In [1]:
import numpy as np
import os

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Embedding, Input
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint

In [3]:
BASE_PATH = "/home/abhai/Workbench/datasets/20news-18828"

In [4]:
label_codes = {}
for index ,label in zip(range(len(os.listdir(BASE_PATH))), sorted(os.listdir(BASE_PATH))):
    label_codes[label] = index

In [5]:
text = []
labels = []
for name in os.listdir(BASE_PATH):
    path = os.path.join(BASE_PATH,name)
    for fname in os.listdir(path):
        f = open(os.path.join(path,fname),'r')
        text.append(f.read())
        labels.append(label_codes[name])
        f.close()

In [6]:
tokenizer = Tokenizer(nb_words=2000)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

In [13]:
data = pad_sequences(sequences,maxlen=1000)

In [14]:
labels = to_categorical(labels,20)
print labels.shape

(18828, 20)


In [15]:
model = Sequential()
# model.add(Input(shape=(100,), dtype='int32'))
model.add(Embedding(len(tokenizer.word_index)+1, 100, input_length=1000))
model.add(Convolution1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Convolution1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Convolution1D(128, 5, activation='relu'))
model.add(MaxPooling1D(35))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(20,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint(filepath="/home/abhai/weights.hdf5", verbose=1, save_best_only=True)
model.fit(data, labels, validation_split=0.1, shuffle=True,batch_size=64, nb_epoch=20, verbose=1, callbacks=[checkpointer])

Train on 16945 samples, validate on 1883 samples
Epoch 1/20
  704/16945 [>.............................] - ETA: 1230s - loss: 1.3863 - acc: 0.6108